In [ ]:
import cv2
import numpy as np
import os
from glob import glob
import matplotlib.pyplot as plt
from skimage.filters import threshold_otsu
from skimage import morphology

# Define paths
input_folder = "/content/ph2"  # Change this to your image folder
output_folder = "/content/ph2_contours"

# Ensure output directories exist
os.makedirs(output_folder, exist_ok=True)

# Process all images in the input folder
for image_path in glob(os.path.join(input_folder, "*.png")):
    # Load image
    img = cv2.imread(image_path)
    if img is None:
        print(f"❌ Skipping {image_path} (Invalid image)")
        continue

    filename = os.path.splitext(os.path.basename(image_path))[0]
    print(f"Processing {filename}...")

    # **Step 1: Convert to Grayscale**
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # **Step 2: Apply Gaussian Blur**
    blurred = cv2.GaussianBlur(gray, (3, 3), 0)  # Small blur to avoid excessive smoothing

    # **Step 3: Apply Otsu's Thresholding**
    thresh_val = threshold_otsu(blurred)
    binary_mask = (blurred < thresh_val).astype(np.uint8) * 255  # Invert mask for lesion

    # **Step 4: Morphological Closing (Fill Small Gaps)**
    binary_mask = morphology.closing(binary_mask, morphology.disk(3)).astype(np.uint8)

    # **Step 5: Morphological Erosion to Shrink Over-segmented Masks**
    binary_mask = morphology.erosion(binary_mask, morphology.disk(2)).astype(np.uint8)

    # Convert binary mask to OpenCV format
    binary_mask_cv = (binary_mask * 255).astype(np.uint8)

    # **Step 6: Extract Contours**
    contours, _ = cv2.findContours(binary_mask_cv, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Create a blank mask for final lesion selection
    final_mask = np.zeros_like(binary_mask_cv)

    if len(contours) > 1:
        # **Check if an Inner Contour Exists**
        lesion_contour = min(contours, key=cv2.contourArea)  # Select smallest valid contour (likely the lesion)
    else:
        # **Keep the only contour if no inner contour is found**
        lesion_contour = max(contours, key=cv2.contourArea)

    # Draw only the correct lesion contour
    cv2.drawContours(final_mask, [lesion_contour], -1, 255, thickness=cv2.FILLED)

    # **Step 7: Prepare Mask for GrabCut**
    grabcut_mask = np.zeros(gray.shape, np.uint8)
    grabcut_mask[final_mask == 255] = cv2.GC_FGD  # Foreground
    grabcut_mask[final_mask == 0] = cv2.GC_BGD  # Background

    # Ensure foreground and background regions exist
    if np.count_nonzero(grabcut_mask == cv2.GC_FGD) == 0:
        print(f"⚠️ No valid foreground detected for {filename}, using rectangle-based GrabCut...")
        rect = (10, 10, gray.shape[1] - 20, gray.shape[0] - 20)  # Bounding box
        cv2.grabCut(img, grabcut_mask, rect, None, None, 5, cv2.GC_INIT_WITH_RECT)
    else:
        # Apply GrabCut with a Mask
        bgd_model = np.zeros((1, 65), np.float64)
        fgd_model = np.zeros((1, 65), np.float64)
        cv2.grabCut(img, grabcut_mask, None, bgd_model, fgd_model, 5, cv2.GC_INIT_WITH_MASK)

    # Convert GrabCut output to a binary mask
    grabcut_output = np.where((grabcut_mask == 2) | (grabcut_mask == 0), 0, 1).astype("uint8")
    grabcut_binary_mask = (grabcut_output * 255).astype(np.uint8)

    # **Step 8: Extract Final Lesion Contour**
    contours, _ = cv2.findContours(grabcut_binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Create a black background for contour visualization
    contour_img = np.zeros_like(img)

    if len(contours) > 0:
        # **Select a contour within a reasonable size range**
        image_area = gray.shape[0] * gray.shape[1]
        min_contour_area = image_area * 0.002  # Minimum size (0.2% of the image area)
        max_contour_area = image_area * 0.8   # Maximum size (20% of the image area)

        valid_contours = [cnt for cnt in contours if min_contour_area < cv2.contourArea(cnt) < max_contour_area]

        if valid_contours:
            lesion_contour = max(valid_contours, key=cv2.contourArea)  # Choose the largest valid contour
            cv2.drawContours(contour_img, [lesion_contour], -1, (0, 255, 0), 2)

    # **Step 9: Overlay Contour on Original Image**
    overlay_img = img.copy()
    if valid_contours:
        cv2.drawContours(overlay_img, [lesion_contour], -1, (0, 255, 0), 2)

    # **Step 10: Save and Display Results**
    final_mask_path = os.path.join(output_folder, filename + "_final_mask.png")
    contour_output_path = os.path.join(output_folder, filename + "_contour.png")
    overlay_output_path = os.path.join(output_folder, filename + "_overlay.png")

    cv2.imwrite(final_mask_path, grabcut_binary_mask)
    cv2.imwrite(contour_output_path, contour_img)
    cv2.imwrite(overlay_output_path, overlay_img)

    print(f"✅ Processed {filename}, results saved.")

    # Display results
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))

    axes[0].imshow(grabcut_binary_mask, cmap='gray')
    axes[0].set_title("Final Binary Mask (Corrected)")
    axes[0].axis("off")

    axes[1].imshow(cv2.cvtColor(contour_img, cv2.COLOR_BGR2RGB))
    axes[1].set_title("Lesion Contour with Correct Size")
    axes[1].axis("off")

    axes[2].imshow(cv2.cvtColor(overlay_img, cv2.COLOR_BGR2RGB))
    axes[2].set_title("Lesion Contour Overlayed on Original Image")
    axes[2].axis("off")

    plt.show()
